#################################################
##      뉴스기사 키워드 추출 및 테마 추천       ##
#################################################

0. 라이브러리 임포트

In [1]:
# -*- coding: utf-8 -*-
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import pickle
import re
import itertools

c:\Users\G-10\anaconda3\envs\tensor\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


1. 뉴스기사 선언

In [2]:
article = """
유한양행의 국산 신약 31호 '렉라자'가 미국 문턱을 넘는데 성공했다.

존슨앤존슨은 20일 미국 식품의약국(FDA)으로부터 리브리반트(성분명 아미반타맙)와 렉라자(성분명 레이저티닙)의 병용요법을 비소세포폐암(NSCLC) 성인 환자의 1차 치료제로 승인받았다고 발표했다. 상피세포 성장인자 수용체(EGFR, 폐암 세포의 성장에 관여) 변이가 있는 국소 진행성 또는 전이성 비소세포폐암 치료에 대해 사용이 가능하다.

이에 따라 유한양행은 존슨앤드존슨으로부터 마일스톤 6000만달러(한화 약 800억7000만원)와 향후 제품 판매에 따른 로열티를 수령하게 된다.

이번 FDA 허가는 1074명의 환자를 대상으로 진행한 '마리포사(MARIPOSA)' 임상3상 연구를 토대로 이뤄졌다. 해당 임상에 따르면 리브리반트와 렉라자 병용요법은 오시머티닙(제품명 타그리소) 단독요법 대비 질병 진행 또는 사망위험을 30% 감소시켰다.

또 무진행 생존기간(PFS)은 오시머티닙의 16.6개월보다 긴 23.7개월로 나타났으며, 위축된 암의 크기가 유지되는 '반응 지속기간(DOR)'도 25.8개월로 오시머티닙의 16.8개월보다 9개월 더 길었다.

이번 FDA 승인으로 유한양행은 R&D에서 오픈이노베이션의 첫 결실을 맺게 됐다. 앞서 유한양행은 2015년 국내 바이오기업 오스코텍으로부터 렉라자를 라이선스인(기술도입)해 국내에서 2021년 허가를 받았다. 이번 FDA 허가를 통해 리브리반트와 렉라자 병용요법이 승인심사를 앞둔 유럽, 중국, 일본 등의 허가에도 긍정적인 영향을 미칠 것으로 보인다. 

폐암은 전 세계적으로 암 사망의 주요 원인으로, 매년 180만명이 사망하며, 비소세포폐암이 80~85%를 차지한다. EGFR 돌연변이 비소세포폐암 환자 중 25~39%는 질병 진행과 치료 옵션 부족으로 인해 2차 치료를 받지 못하는 상황이다.

현재 표준 치료인 TKI 단일 요법으로 치료하는 진행성 EGFR 돌연변이 비소세포폐암 환자의 5년 생존율은 20% 미만이다. 이번 리브리반트와 렉라자 병용요법 승인으로 그동안 오시머티닙 치료에만 의존해야 했던 환자들에게 새로운 치료요법이 확대된 셈이다. 

존슨앤드존슨 제니퍼 톨버트(Jennifer Taubert) 제약부문 회장은 "리브리반트와 렉라자 병용요법은 EGFR 변이 1차 치료에 새로운 기준을 확립했다"고 했다.

글로벌 비소세포폐암 치료제 시장규모는 지난 2022년 약 30조원으로, 이 중 비소세포폐암 EGFR 변이 치료제 시장규모는 약 5조원에 달한다. 현재 오시머티닙이 EGFR 변이 치료제 시장의 50% 이상을 차지하고 있다. 임상에서 리브리반트와 렉라자 병용요법이 우월성을 입증한 만큼 시장 판도도 리브리반트와 렉라자 병용요법으로 기울 것으로 기대를 모은다.

유한양행 조욱제 사장은 "렉라자의 FDA의 승인은 오픈이노베이션을 통한 유한양행 R&D투자의 유의미한 결과물"이라며 "이번 승인이 종착점이 아닌 하나의 통과점이 돼 R&D 투자를 통해 한국을 대표하는 글로벌 혁신신약 출시와 함께 유한양행의 글로벌 톱50 기업 달성을 위한 초석이 되길 기대한다"고 강조했다.
"""

2. 텍스트 전처리

In [3]:
article = article.replace('\n', '')
split_article = re.split(r'(\s*[a-zA-Z]+(?:[^\w\s]+[a-zA-Z]+)*\s*)', article)       #   한글과 영어 나누기
split_article = [part.strip() for part in split_article if part.strip()]            #   불필요한 공백 지우기

okt = Okt()                         #   형태소 분석기 Okt
nouns=[]
for sentence in split_article:
    sen = okt.nouns(sentence)
    if len(sen) != 0:               #   명사를 리스트에 추가
        nouns.extend(sen)
    else:
        nouns.append(sentence)      #   영어도 리스트에 추가(형태소 분석 후 길이가 0이면 영어로 판단)

text = ' '.join(nouns)              #   리스트를 하나의 문자열로

print(f'전처리 후 뉴스 기사 : {text}')

전처리 후 뉴스 기사 : 유한양행 국산 신약 호 렉 자 미국 문턱 존슨앤존슨 미국 식품 약국 FDA 리브 리 트 성분 아미 맙 렉 자 성분 레이저 티닙 의 병용 요법 비소 세포 폐암 NSCLC 성인 환자 차 치료 제로 승인 발표 상피세포 성장 인자 수용체 EGFR 폐암 세포 성장 관여 변 국소 진행 이성 비소 세포 폐암 치료 대해 사용 이 유한양행 존슨 앤드 존슨 마일 스톤 한화 약 향후 제품 판매 로열티 수령 이번 FDA 허가 명의 환자 대상 진행 마리 포사 MARIPOSA 임 상 연구 토대 해당 임 리브 리 트 렉 자 병용 요법 머티닙 제품 타 단독 요법 대비 질병 진행 사망 위험 감소 또 무진행 생존 기간 PFS 은 머티닙 개 월보 개월 위축 암 크기 유지 반응 지속 기간 DOR 도 개월 머티닙 개 월보 개월 더 이번 FDA 승인 유한양행 R&D 오픈 이노베이션 첫 결실 앞서 유한양행 국내 바이오 기업 오스코텍 렉 라이선스 기술 도입 해 국내 허가 이번 FDA 허가 통해 리브 리 트 렉 자 병용 요법 승인 심사 유럽 중국 일본 등 허가 긍정 영향 것 폐암 전 세계 암 사망 주요 원인 매년 사망 비소 세포 폐암 를 차지 EGFR 돌연변이 비소 세포 폐암 환자 중 질병 진행 치료 옵션 부족 차 치료 상황 현재 표준 치료 TKI 단일 요법 치료 진행 EGFR 돌연변이 비소 세포 폐암 환자 생존 율 미만 이번 리브 리 트 렉 자 병용 요법 승인 그동안 머티닙 치료 의존 환자 치료 요법 확대 셈 존슨 앤드 존슨 제니퍼 톨버트 Jennifer Taubert 제약 부문 회장 리브 리 트 렉 자 병용 요법 EGFR 변 차 치료 기준 확립 고 글로벌 비소 세포 폐암 치료 제 시장 규모 지난 약 이 중 비소 세포 폐암 EGFR 변 치료 제 시장 규모 약 달 현재 머티닙 EGFR 변 치료 제 시장 이상 차지 임 리브 리 트 렉 자 병용 요법 우월 입증 만큼 시장 판도 리브 리 트 렉 자 병용 요법 것 기대 유한양행 조욱 제 사장 렉 자의 FDA 의 승인 오픈 이노베이션 통한

3. N-Gram 생성 및 SBERT

In [4]:
n_gram_range = (1,2)
word_vectorize = CountVectorizer(ngram_range=n_gram_range).fit([text])   #   1 ~ 2 글자로 N-Gram 생성
n_gram_words = word_vectorize.get_feature_names_out()
print(f'N-Gram 개수 : {len(n_gram_words)}')
print(f'N-Gram 다섯개 출력 : {n_gram_words[:5]}')

model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')  #   Sentence Transformers
article_embedding = model.encode([article])
n_gram_embeddings = model.encode(n_gram_words)
print(f'SBERT 분석 후 뉴스 기사 : {article_embedding}')
print(f'SBERT 분석 후 N-Gram : {n_gram_embeddings}')

N-Gram 개수 : 375
N-Gram 다섯개 출력 : ['dor' 'dor 개월' 'egfr' 'egfr 돌연변이' 'egfr 치료']


c:\Users\G-10\anaconda3\envs\tensor\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


SBERT 분석 후 뉴스 기사 : [[-5.33791542e-01 -3.12049091e-01  1.16301022e-01 -4.80113119e-01
   6.16123438e-01 -1.39521313e+00  2.26116747e-01 -1.96823701e-01
   6.77545974e-03 -1.99713141e-01 -1.32318974e-01  4.06395793e-01
   2.39658773e-01  2.53966600e-01 -1.70458049e-01  1.27820373e-01
  -2.69211829e-01  4.41016704e-01  3.87374490e-01 -2.06353962e-01
  -2.48334572e-01 -8.25921237e-01  1.12514123e-01  7.78784513e-01
   9.32966769e-01 -4.43295296e-03  6.55870795e-01 -9.21325907e-02
   6.32999718e-01  5.65240026e-01 -8.27533975e-02 -7.48881817e-01
  -3.80383044e-01 -1.14131081e+00  5.24293445e-03  9.09790635e-01
  -1.97767898e-01  6.13062680e-01  6.56779647e-01 -1.66133627e-01
  -5.53524733e-01 -3.90226841e-01 -3.99335437e-02 -6.40735030e-01
  -6.69691205e-01 -6.96270838e-02 -5.11808813e-01  2.41456747e-01
   3.96177083e-01  1.89410057e-04  3.07444096e-01 -1.94400981e-01
   8.85654241e-04 -4.34411407e-01 -1.99702576e-01  4.24345315e-01
  -3.63609314e-01  3.92954111e-01 -8.12281847e-01  4.2466

4. Max Sum Similarity

In [5]:
def max_sum_sim(article_embedding, n_gram_embeddings, n_gram_words, top_n, variety):
    #   뉴스 기사와 N-Gram의 유사도
    distances = cosine_similarity(article_embedding, n_gram_embeddings)
    #   N-Gram들 사이의 유사도
    distances_candidates = cosine_similarity(n_gram_embeddings, n_gram_embeddings)

    #   뉴스 기사와 유사도가 높은 N-Gram을 variety개 가져오기
    words_idx = list(distances.argsort()[0][-variety:])
    words_vals = [n_gram_words[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    #   그 중 유사도가 가장 낮은 조합 찾기
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

news_keywords = max_sum_sim(article_embedding, n_gram_embeddings, n_gram_words, top_n=5, variety=10)
print(f'뉴스 키워드 5개 : {news_keywords}')

뉴스 키워드 5개 : ['신약 미국', '상피세포 성장', 'egfr 폐암', '폐암 치료', '신약 출시']


5. 테마 가져오기

In [6]:
#       딕셔너리에서 key 값 가져오기
with open('D:/TJ_FInal_Project/KDJ/News_Keyword_Theme_Recommend.py/data/theme_dict.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

keys = loaded_dict.keys()
keys_list = list(keys)

new_keys_list = [re.sub(r'[^a-zA-Z0-9가-힣\s]', ' ', key) for key in keys_list]     #   테마 전처리
key_embeddings = model.encode(new_keys_list)                                        #   테마  SBERT 분석

6. 코사인 거리 기반 & Max Sum Similarity 테마 추천

In [7]:
top_n = 5
distances = cosine_similarity(article_embedding, key_embeddings)
cosine_recommend = [keys_list[index] for index in distances.argsort()[0][-top_n:]]

# maxsum_recommend = max_sum_sim(article_embedding, key_embeddings, keys_list, top_n=3, variety=30)

article = article.replace('.', '.\n')
print(f'뉴스 기사 : \n{article}')
print(f'뉴스 키워드 5개 : {news_keywords}')

print(f'\n뉴스 기사와 연관 되어 있는 테마 5개')
for i in range(0,5):
    print(f'{i+1}. {cosine_recommend[-i]} : {loaded_dict[cosine_recommend[-i]]}')

# print(f'\n뉴스 기사와 연관될 수 있는 테마 3개')
# maxsum_index = 1
# for maxsum in maxsum_recommend:
#     print(f'{maxsum_index}. {maxsum} : {loaded_dict[maxsum]}')
#     maxsum_index += 1

뉴스 기사 : 
유한양행의 국산 신약 31호 '렉라자'가 미국 문턱을 넘는데 성공했다.
존슨앤존슨은 20일 미국 식품의약국(FDA)으로부터 리브리반트(성분명 아미반타맙)와 렉라자(성분명 레이저티닙)의 병용요법을 비소세포폐암(NSCLC) 성인 환자의 1차 치료제로 승인받았다고 발표했다.
 상피세포 성장인자 수용체(EGFR, 폐암 세포의 성장에 관여) 변이가 있는 국소 진행성 또는 전이성 비소세포폐암 치료에 대해 사용이 가능하다.
이에 따라 유한양행은 존슨앤드존슨으로부터 마일스톤 6000만달러(한화 약 800억7000만원)와 향후 제품 판매에 따른 로열티를 수령하게 된다.
이번 FDA 허가는 1074명의 환자를 대상으로 진행한 '마리포사(MARIPOSA)' 임상3상 연구를 토대로 이뤄졌다.
 해당 임상에 따르면 리브리반트와 렉라자 병용요법은 오시머티닙(제품명 타그리소) 단독요법 대비 질병 진행 또는 사망위험을 30% 감소시켰다.
또 무진행 생존기간(PFS)은 오시머티닙의 16.
6개월보다 긴 23.
7개월로 나타났으며, 위축된 암의 크기가 유지되는 '반응 지속기간(DOR)'도 25.
8개월로 오시머티닙의 16.
8개월보다 9개월 더 길었다.
이번 FDA 승인으로 유한양행은 R&D에서 오픈이노베이션의 첫 결실을 맺게 됐다.
 앞서 유한양행은 2015년 국내 바이오기업 오스코텍으로부터 렉라자를 라이선스인(기술도입)해 국내에서 2021년 허가를 받았다.
 이번 FDA 허가를 통해 리브리반트와 렉라자 병용요법이 승인심사를 앞둔 유럽, 중국, 일본 등의 허가에도 긍정적인 영향을 미칠 것으로 보인다.
 폐암은 전 세계적으로 암 사망의 주요 원인으로, 매년 180만명이 사망하며, 비소세포폐암이 80~85%를 차지한다.
 EGFR 돌연변이 비소세포폐암 환자 중 25~39%는 질병 진행과 치료 옵션 부족으로 인해 2차 치료를 받지 못하는 상황이다.
현재 표준 치료인 TKI 단일 요법으로 치료하는 진행성 EGFR 돌연변이 비소세포폐암 환자의 5년 생존율은 20% 미만이다.
 이번 